<a href="https://colab.research.google.com/github/mariobecerra/mda_project/blob/main/code/01_weekly_temperature_belgium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

URLs of data

In [7]:
arrondissements_url = 'https://raw.githubusercontent.com/mariobecerra/mda_project/main/data/arrondissements_coords.csv'
mortality_url = 'https://raw.githubusercontent.com/mariobecerra/mda_project/main/data/mortality_data_2000-2019.csv'
temperature_url = 'https://raw.githubusercontent.com/mariobecerra/mda_project/main/data/temp_2000_2019.csv'


Read data

In [8]:
import pandas as pd

In [9]:
arrondissements_data = pd.read_csv(arrondissements_url)

arrondissements_data.head(5)

,NIS_Code,Nom_arrondissement,lat,lon
0,11000,Antwerpen,51.280681,4.505809
1,63000,Verviers,50.464213,6.022070
2,13000,Turnhout,51.250094,4.950422
3,62000,Liège,50.613433,5.607778
4,64000,Waremme,50.662789,5.206501


In [12]:
mortality_data = pd.read_csv(mortality_url)
mortality_data.head(15)


,YEAR,YEAR_WEEK,COD,ARRON,N_MASK
0,2000,2000-001,external,11000,NaN
1,2000,2000-001,external,12000,NaN
2,2000,2000-001,external,13000,NaN
3,2000,2000-001,external,21000,NaN
4,2000,2000-001,external,23000,NaN
5,2000,2000-001,external,24000,5.0
6,2000,2000-001,external,31000,NaN
7,2000,2000-001,external,34000,NaN
8,2000,2000-001,external,35000,NaN
9,2000,2000-001,external,44000,NaN


In [11]:
temperature_data = pd.read_csv(temperature_url, sep=";")
temperature_data.head(5)

,GRID_NO,LATITUDE,LONGITUDE,DAY,TEMPERATURE_MAX,TEMPERATURE_MIN,TEMPERATURE_AVG
0,96095,49.59159,5.03811,20000101,6.2,2.9,4.5
1,96095,49.59159,5.03811,20000102,6.0,4.2,5.1
2,96095,49.59159,5.03811,20000103,5.4,3.1,4.3
3,96095,49.59159,5.03811,20000104,8.1,4.4,6.3
4,96095,49.59159,5.03811,20000105,8.1,1.8,4.9
